### Import libraries and create DataFrame from csv 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Function to read, preprocess, and aggregate full season data
def read_data():
    # Read the full season data
    data = pd.read_excel("QB_Data_2024.xlsx")

    # Preprocess full season data, drop irrelevant columns and convert data types
    data.drop(['FL', 'ROST'], axis=1, inplace=True)
    convert = data.select_dtypes('object').columns.difference(['Player'])
    data[convert] = data[convert].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0)
    data['Rank'] = data['Rank'].astype('Int64')

    return data

# Load the full season data
df = read_data()

# Recalculate 'Rank' based on 'FPTS' in descending order
df['Rank'] = df['FPTS'].rank(ascending=False, method='min').astype('int')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')
df.head(20)

,Rank,Player,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT.1,YDS.1,TD.1,G,FPTS,FPTS/G
0,1,Dak Prescott (DAL),96,149,64.4,1072,7.2,6,2,10,7,21,1,4,73.0,18.3
1,2,Josh Allen (BUF),54,72,75.0,634,8.8,7,0,2,17,85,2,3,71.9,24.0
2,3,Jayden Daniels (WAS),61,76,80.3,664,8.7,2,0,9,38,171,3,3,69.7,23.2
3,4,Lamar Jackson (BAL),59,90,65.6,702,7.8,3,1,3,35,254,1,3,68.5,22.8
4,5,Sam Darnold (MIN),53,78,67.9,657,8.4,8,2,8,8,35,0,3,59.7,19.9
5,6,Baker Mayfield (TB),61,82,74.4,637,7.8,6,2,13,8,55,1,3,59.0,19.7
6,7,Kyler Murray (ARI),59,86,68.6,635,7.4,5,1,6,15,161,0,3,58.5,19.5
7,8,Jalen Hurts (PHI),72,102,70.6,772,7.6,3,4,7,34,143,1,3,55.1,18.4
8,8,Daniel Jones (NYG),91,144,63.2,881,6.1,4,3,9,23,70,0,4,55.1,13.8
9,10,Derek Carr (NO),44,64,68.8,585,9.1,6,2,3,5,17,1,3,53.1,17.7


### Convert relevant stats to a per-game basis and create the final stats for analysis

In [2]:
# Define columns for per-game calculation
per_game = list(df.columns[2:])

# Define columns to exclude from the per-game calculation
exclude_per_game = ['PCT', 'Y/A', 'G', 'FPTS', 'FPTS/G']

# Create columns and convert stats to a per-game basis for the defined columns, not those in the exclude list
for col in per_game:
    if col not in exclude_per_game:
        df[col + '/game'] = (df[col] / df['G']).round(1)

# Define final columns to be used for analysis, combining excluded and new per-game columns
final_columns = exclude_per_game + [col + '/game' for col in per_game if col not in exclude_per_game]

# Display final columns with 'Rank' and 'Player'
df[['Rank', 'Player'] + final_columns].head(10)

,Rank,Player,PCT,Y/A,G,FPTS,FPTS/G,CMP/game,ATT/game,YDS/game,TD/game,INT/game,SACKS/game,ATT.1/game,YDS.1/game,TD.1/game
0,1,Dak Prescott (DAL),64.4,7.2,4,73.0,18.3,24.0,37.2,268.0,1.5,0.5,2.5,1.8,5.2,0.2
1,2,Josh Allen (BUF),75.0,8.8,3,71.9,24.0,18.0,24.0,211.3,2.3,0.0,0.7,5.7,28.3,0.7
2,3,Jayden Daniels (WAS),80.3,8.7,3,69.7,23.2,20.3,25.3,221.3,0.7,0.0,3.0,12.7,57.0,1.0
3,4,Lamar Jackson (BAL),65.6,7.8,3,68.5,22.8,19.7,30.0,234.0,1.0,0.3,1.0,11.7,84.7,0.3
4,5,Sam Darnold (MIN),67.9,8.4,3,59.7,19.9,17.7,26.0,219.0,2.7,0.7,2.7,2.7,11.7,0.0
5,6,Baker Mayfield (TB),74.4,7.8,3,59.0,19.7,20.3,27.3,212.3,2.0,0.7,4.3,2.7,18.3,0.3
6,7,Kyler Murray (ARI),68.6,7.4,3,58.5,19.5,19.7,28.7,211.7,1.7,0.3,2.0,5.0,53.7,0.0
7,8,Jalen Hurts (PHI),70.6,7.6,3,55.1,18.4,24.0,34.0,257.3,1.0,1.3,2.3,11.3,47.7,0.3
8,8,Daniel Jones (NYG),63.2,6.1,4,55.1,13.8,22.8,36.0,220.2,1.0,0.8,2.2,5.8,17.5,0.0
9,10,Derek Carr (NO),68.8,9.1,3,53.1,17.7,14.7,21.3,195.0,2.0,0.7,1.0,1.7,5.7,0.3


### Calculate the correlations for the final stats across different conditions

In [3]:
# Define columns and columns to exclude for correlation calculation, excluding FPTS and FPTS/G
exclude_corr = ['FPTS/G', 'FPTS', 'G']
corr_columns = [col for col in final_columns if col not in exclude_corr]

# Define a function to calculate correlations
def compute_correlations(dataframe, corr_columns):
    return dataframe[corr_columns].corrwith(dataframe['FPTS/G'])

# Compute correlations for various conditions
corr_all = compute_correlations(df, corr_columns)
corr_nonzero = compute_correlations(df[df['FPTS/G'] > 0], corr_columns)
corr_top50 = compute_correlations(df[df['Rank'] <= 50], corr_columns)
corr_top25 = compute_correlations(df[df['Rank'] <= 25], corr_columns)

# Compile all correlations into a DataFrame for comparison
df_corr = pd.DataFrame({
    'All Players': corr_all,
    'FPTS > 0': corr_nonzero,
    'Top 50 Players': corr_top50,
    'Top 25 Players': corr_top25
})

# Calculate the average correlation across the four conditions, adding 'Average' column to DataFrame
df_corr['Correlation'] = df_corr.mean(axis=1)

# Display the correlation DataFrame
df_corr.round(2)

,All Players,FPTS > 0,Top 50 Players,Top 25 Players,Correlation
PCT,0.83,0.40,0.87,0.32,0.61
Y/A,0.89,0.55,0.90,0.46,0.70
CMP/game,0.89,0.77,0.87,-0.02,0.63
ATT/game,0.86,0.70,0.84,-0.16,0.56
YDS/game,0.93,0.85,0.92,0.28,0.75
TD/game,0.85,0.75,0.83,0.42,0.71
INT/game,0.51,0.30,0.46,-0.47,0.20
SACKS/game,0.57,0.20,0.50,-0.17,0.28
ATT.1/game,0.65,0.59,0.64,0.46,0.59
YDS.1/game,0.64,0.57,0.60,0.48,0.57


### Assign the weights for the final stats

In [4]:
# Calculate R^2 for the 'Average' correlation, adding 'R^2' column to DataFrame
df_corr['R^2'] = df_corr['Correlation'] ** 2

# Select stats with R^2 above a threshold for higher weight
high_weight_threshold = 0.5
specific_stats = df_corr[df_corr['R^2'] > high_weight_threshold].index.tolist()

# Select stats with R^2 below a threshold to exclude from final score
exclude_threshold = 0.1
exclude_stats = df_corr[df_corr['R^2'] < exclude_threshold].index.tolist()

# Define the calculation to assign weights
def weight_calc(row, specific_stats):
    if row.name in specific_stats:
        return 1 + row['R^2'] * 2 # Assign higher weight to specific stats
    else:
        return 1 + row['R^2'] # Assign weight to all other stats
    
# Assign weights based on the given criteria, adding 'Weight' column to DataFrame
df_corr['Weight'] = df_corr.apply(weight_calc, specific_stats=specific_stats, axis=1)

# Normalize weights
df_corr['Weight'] = df_corr['Weight'] / df_corr['Weight'].sum()

# Display the new columns in the DataFrame
df_corr[['Correlation', 'R^2', 'Weight']].round(2)

,Correlation,R^2,Weight
PCT,0.61,0.37,0.09
Y/A,0.70,0.49,0.09
CMP/game,0.63,0.39,0.09
ATT/game,0.56,0.31,0.08
YDS/game,0.75,0.56,0.13
TD/game,0.71,0.51,0.13
INT/game,0.20,0.04,0.07
SACKS/game,0.28,0.08,0.07
ATT.1/game,0.59,0.34,0.09
YDS.1/game,0.57,0.33,0.08


### Multiply the assigned weights to the final stats and calculate the score

In [5]:
# Multiply each relevant column by its corresponding weight
for col in corr_columns:
    if col not in exclude_stats:
        weight = df_corr.loc[col, 'Weight']
        df[col + '_weighted'] = (df[col] * weight).round(1)

# Extract the weighted columns
weight_columns = [col + '_weighted' for col in corr_columns if col not in exclude_stats]

# Display the new weighted stats columns
df_weight = df[['Rank', 'Player', 'FPTS/G'] + weight_columns]
df_weight.sort_values(by=['Rank'], ascending=True).head(10)

# Define select columns to be used for the average weighted score
avg = weight_columns + (['FPTS/G'] * 2)  # Giving higher weight to FPTS/G

# Calculate the average weighted score for the select columns
df['Score'] = df[avg].mean(axis=1).round(2)

# Normalize the scores to be out of 10
scaler = MinMaxScaler(feature_range=(0, 10))
df['Score'] = scaler.fit_transform(df[['Score']])

df_weight.head(10)

,Rank,Player,FPTS/G,PCT_weighted,Y/A_weighted,CMP/game_weighted,ATT/game_weighted,YDS/game_weighted,TD/game_weighted,ATT.1/game_weighted,YDS.1/game_weighted,TD.1/game_weighted
0,1,Dak Prescott (DAL),18.3,5.6,0.7,2.1,3.1,36.0,0.2,0.2,0.4,0.0
1,2,Josh Allen (BUF),24.0,6.5,0.8,1.6,2.0,28.4,0.3,0.5,2.4,0.1
2,3,Jayden Daniels (WAS),23.2,7.0,0.8,1.8,2.1,29.7,0.1,1.1,4.8,0.1
3,4,Lamar Jackson (BAL),22.8,5.7,0.7,1.7,2.5,31.4,0.1,1.0,7.1,0.0
4,5,Sam Darnold (MIN),19.9,5.9,0.8,1.6,2.2,29.4,0.3,0.2,1.0,0.0
5,6,Baker Mayfield (TB),19.7,6.5,0.7,1.8,2.3,28.5,0.3,0.2,1.5,0.0
6,7,Kyler Murray (ARI),19.5,6.0,0.7,1.7,2.4,28.4,0.2,0.4,4.5,0.0
7,8,Jalen Hurts (PHI),18.4,6.1,0.7,2.1,2.8,34.5,0.1,1.0,4.0,0.0
8,8,Daniel Jones (NYG),13.8,5.5,0.6,2.0,3.0,29.6,0.1,0.5,1.5,0.0
9,10,Derek Carr (NO),17.7,6.0,0.9,1.3,1.8,26.2,0.3,0.1,0.5,0.0


### Model training

In [6]:
# Prepare features and target for model training
X = df[weight_columns]
y = df['Score']

# Handle missing values by imputing with mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Initialize Ridge regression model
ridge_model = Ridge()

# Cross-validation to evaluate the model
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation MSE: {-cv_scores.mean()}')

# Train the model
ridge_model.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Set MSE: {mse}')

# Incorporate Random Forest as an ensemble method
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest Test Set MSE: {rf_mse}')

# Average predictions from both models for final score
final_pred = (y_pred + rf_pred) / 2

# Create a DataFrame for the test set predictions
test_results = pd.DataFrame({'Final_Score': final_pred}, index=test_idx)

# Merge the test set predictions back into the original DataFrame
df = df.merge(test_results, how='left', left_index=True, right_index=True)

# Fill NaN values in 'Final_Score' column with the original 'Score' to handle missing indices
df['Final_Score'].fillna(df['Score'], inplace=True)

# Normalize final scores to be out of 10
df['Final_Score'] = scaler.fit_transform(df[['Final_Score']]).round(2)

# Rank the final scores
df['Final Rank'] = df['Final_Score'].rank(method='first', ascending=False).astype(int)

# Calculate the variance in ranking both ranks
df['Variance'] = df['Rank'] - df['Final Rank']

Cross-Validation MSE: 1.0215618610179888
Test Set MSE: 1.8603217930640188
Random Forest Test Set MSE: 0.28693699425714014


/tmp/ipykernel_9569/591390400.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Final_Score'].fillna(df['Score'], inplace=True)


### Display final results and export to Excel

In [7]:
# List of columns to be excluded
final_columns_exclude = ['Y/R', 'LG', 'ATT/game', 'YDS.1/game', 'TD.1/game']

# Exclude the specified columns from final_columns
final_columns = [col for col in final_columns if col not in final_columns_exclude]

# Create final analysis columns
analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + final_columns]
analysis.set_index('Rank', inplace=True)
analysis = analysis.sort_values(by='Final Rank', ascending=True)

# Export to Excel
analysis.to_excel("QB_Analysis.xlsx", index=False)

# Display the top 30 rows
analysis.head(30)

,Final Rank,Player,Final_Score,Variance,PCT,Y/A,G,FPTS,FPTS/G,CMP/game,YDS/game,TD/game,INT/game,SACKS/game,ATT.1/game
Rank,,,,,,,,,,,,,,,
4,1,Lamar Jackson (BAL),10.00,3,65.6,7.8,3,68.5,22.8,19.7,234.0,1.0,0.3,1.0,11.7
3,2,Jayden Daniels (WAS),9.82,1,80.3,8.7,3,69.7,23.2,20.3,221.3,0.7,0.0,3.0,12.7
2,3,Josh Allen (BUF),9.50,-1,75.0,8.8,3,71.9,24.0,18.0,211.3,2.3,0.0,0.7,5.7
8,4,Jalen Hurts (PHI),9.25,4,70.6,7.6,3,55.1,18.4,24.0,257.3,1.0,1.3,2.3,11.3
7,5,Kyler Murray (ARI),8.79,2,68.6,7.4,3,58.5,19.5,19.7,211.7,1.7,0.3,2.0,5.0
6,6,Baker Mayfield (TB),8.58,0,74.4,7.8,3,59.0,19.7,20.3,212.3,2.0,0.7,4.3,2.7
11,7,Brock Purdy (SF),8.41,4,72.6,8.9,3,51.1,17.0,23.0,280.7,1.3,0.3,3.0,4.3
33,8,Jordan Love (GB),8.41,25,50.0,7.6,1,17.4,17.4,17.0,260.0,2.0,1.0,1.0,0.0
13,9,Geno Smith (SEA),8.04,4,74.8,7.6,3,50.1,16.7,25.7,262.3,1.0,1.0,2.7,3.7
